# SpeechRecognition Project 
* Only transcribed with single language for each file (until now)
* For transcription, nearly 90% successful.
* English, spoken by Korean children is recognized rather as Korean not as English.
* If the file is too long(over 10 minutes), it might be better to upload the audio file manually.
* 3gp file is converted to wav file for the experiment.

In [1]:
import speech_recognition as sr

filepath = "audio/"     #Input audio file path in my case
output_filepath = "Transcripts/" # Final transcript path
bucketname = "speechrecognitionkt" # Name of the bucket created in the step before

# Import libraries
from pydub import AudioSegment
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import wave
from google.cloud import storage

In [2]:
def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")

In [3]:
def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

In [9]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    if not blob.exists():
        blob.upload_from_filename(source_file_name)

In [5]:
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [17]:
def google_transcribe(audio_file_name):
    
    file_name = filepath + audio_file_name
#     mp3_to_wav(file_name)

    # The name of the audio file to transcribe
    
    frame_rate, channels = frame_rate_channel(file_name)
    
    if channels > 1:
        stereo_to_mono(file_name)
    
    bucket_name = bucketname
    source_file_name = filepath + audio_file_name
    destination_blob_name = audio_file_name
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://' + bucketname + '/' + audio_file_name
    transcript = ''
    
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='ko-KR')

    # Detects speech in the audio file
    operation = client.long_running_recognize(config, audio)
    response = operation.result(timeout=10000)

    for result in response.results:
        transcript += result.alternatives[0].transcript
    
    delete_blob(bucket_name, destination_blob_name)
    return transcript

In [7]:
def write_transcripts(transcript_filename,transcript):
    f= open(output_filepath + transcript_filename,"w+")
    f.write(transcript)
    f.close()

## Example audio clip

In [1]:
recognizer = sr.Recognizer()
recognizer.energy_threshold = 300

harvard_audio = sr.AudioFile("audio/harvard.wav")

with harvard_audio as source:
    audio = recognizer.record(source)
    
recognizer.recognize_google(audio_data = audio, language = "en-US")

'the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle taste fine with ham tacos al Pastore are my favorite a zestful food is be hot cross bun'

## 전사.3gp (-> wav 파일로 변환 후 테스트)

In [18]:
test_transcript = google_transcribe('test.wav')
print(test_transcript)

야 뭐 하고 있어 길게 안 해도 돼 짧게 짧게 세수하고 양치하고 있고요 밥 먹고 옷 입고 저거 영어로 시라고 할게 했었지이따가 된다는 당연히이 같이 가지 기억이 안 나 이런 영어는 안 먹는다며 기억이 안 나 요금 나한테 나는 라면 뭔데 먹는다는 되게 쳐져 이거 빨리 마치는 시간 내야 되니까 먹는다는 이것만 마시자 밥 먹는다는 25분 신발 신을 다녀


## 이해.3gp (한국어로 인식)

In [19]:
understand_transcript = google_transcribe('understand.wav')
print(understand_transcript)

3시 딱 맞아요 맞아요 들려요 맞아요 싫어요 맞아요 틀려요 틀려요 들려 맞아요 틀려요 맞아요 맞아요 틀려요 들려요 맞아요 졸려 들려요 맞아요 틀려요 맞아요 틀려요 들려요 아 맞아요 맞아요 맞아요 맞아 아 맞아 틀려 맞아요 들려요 여기서 또 있잖아


## 발화.3gp (한국어로 인식)

In [7]:
balhwa = sr.AudioFile("audio/balhwa.wav")

with balhwa as source:
    balhwa_source = recognizer.record(source)
    
recognizer.recognize_google(audio_data = balhwa_source, language = "ko-KR")

'도현이네 스톤 플라워로드 파이어 ccm13 클래스 트리 10시 1분 40분 있다 집중 스페이스 마우스 마우스 마우스 싱글'

In [10]:
balhwa_transcript = google_transcribe('balhwa.wav')
print(balhwa_transcript)

도현이네 스톤 스카이 플라워로드썬 파이어 c c c 클래스 타 트리 버드 워터 10시 1분 집중 lose your face 핸드폰 케이스 마우스 마우스 I 빙글


# 1차시수업.mp4 -> lesson.wav 변환 후 한국어 인식

In [32]:
transcript = google_transcribe('lesson.wav')
print(transcript)

안녕하세요 여러분 이번 학기 영화 결석을 많이 드는 비결입니다 교실에서 여러분의 듣고 싶은데 코로나 19 차 때문에서 여러분을 교실에서 뵙지 못해서 너무 안타깝습니다 take a가 빨리 끝내서 여러분을 교실에서 뵙기를 희망합니다 먼저 출석을 부르겠습니다 제가 출석부를때 여러분 이름이 들이지 않는다면 신청이 안 된 거 실수가 있으니 확인해 보시면 되겠습니다 이지윤 있으나 장세원 신현주 김민철 병원 신건호 조현서 박민경 신윤건 오승윤 이서연 주하연 조윤서 홍성현 권가현 문가빈 김도 민선웅 김은강 이다은 최윤지 김익성 이영표 송민준 임수아 강수민 조은비 안세연 권석 총 39명나 이제 수업을 시작하겠습니다 이번 학기 제기되면 수업은 제가 etr 공주에서 올린것처럼 한주의 동영상 만들어서 제가 이태리 올리는 방식으로 진행하려고 합니다 그리고 여러분들에게 간단한 과제를 내려고 합니다 그리고 이번 주에 과제는 제가 동영상 수업 중간에 알려 드리겠습니다 자 오늘은 저희가 2학기에 소속이기 때문에 먼저 쓰려고 스를 소개하고 영어 아기랑 감옥이 무엇인지에 대해서 간단하게 설명을 드리려고 합니다 그리고 인간 언어가 왜 특별한 씨에 대해서 역시 제가 간단하게 설명드리겠습니다 보시는 것이 좋습니다 달라고 했을 때 보시면 맨 위에 강희자 정보 보이시죠 제기 테킬라 관계자는 김기택 저희 고요 그 옆에 이메일 주소가 있습니다장비 밑에 크리스티안 해 보겠습니다 자 여기 뭐 이것저것 내용이 적혀 있는데요이 내용을 간단히 요약하자면 이용확인 원이란 사무기 언어를 이해하는 건 또 그런 어 중에서 특별히 영화의 수정을 드는 감옥 있다라고 이해하시면 될 거 같고요 그리고 여러분들이이 수업을 통해서 확진자 끝날 때 쯤에는 우리 인간의 언어가 만화 놀랐고 그리고 정교한 세 개를 가지고 있는지 그리고이 정교하고 놀라운 복잡한 세계 언어 스택 한다는 것이 얼마나 놀라운 일인지를 이해하게 될 것이라고 생각합니다 그리고 또 난 이런 인간 에어 너야말로 우리를 다른 동물들 예를 들어서 침팬지나 먹었다는 동물들과